# PRE-PROCESSING et MACHINE LEARNING

## Importer les librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

df_clean=pd.read_csv("df_clean_v2.csv", index_col=0)
df_clean.head()

,Platform,Year,Genre,Publisher,Global_Sales,Meta_NP,Meta_VP,Meta_NUsers,Meta_VUsers,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,Support_JVC,langue_parlée_JVC,texte_JVC,freq_name
0,NES,1985,Platform,Nintendo,40.24,84.0,14.0,8.4,119.0,19.0,3.0,19.0,3.0,Autre,français,français,2.0
1,GB,1989,Puzzle,Nintendo,30.26,65.0,10.0,8.0,7.0,11.0,8.0,17.8,3.0,Autre,français,français,4.0
2,Wii,2006,Misc,Nintendo,29.02,58.0,41.0,6.6,190.0,12.0,223.0,12.1,3.0,DVD,français,français,4.0
3,DS,2005,Simulation,Nintendo,24.76,83.0,54.0,7.6,125.0,15.0,159.0,16.4,3.0,Cartouche,français,français,4.0
4,Wii,2007,Sports,Nintendo,22.72,80.0,63.0,7.7,164.0,15.0,227.0,16.2,3.0,DVD,français,français,3.0


In [2]:
df_gd=pd.get_dummies(df_clean)
df_gd.head()

,Year,Global_Sales,Meta_NP,Meta_VP,Meta_NUsers,Meta_VUsers,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,...,Support_JVC_Autre,Support_JVC_Cartouche,Support_JVC_DVD,Support_JVC_PlayStation Store,langue_parlée_JVC_anglais,langue_parlée_JVC_français,langue_parlée_JVC_japonais,texte_JVC_anglais,texte_JVC_français,texte_JVC_japonais
0,1985,40.24,84.0,14.0,8.4,119.0,19.0,3.0,19.0,3.0,...,True,False,False,False,False,True,False,False,True,False
1,1989,30.26,65.0,10.0,8.0,7.0,11.0,8.0,17.8,3.0,...,True,False,False,False,False,True,False,False,True,False
2,2006,29.02,58.0,41.0,6.6,190.0,12.0,223.0,12.1,3.0,...,False,False,True,False,False,True,False,False,True,False
3,2005,24.76,83.0,54.0,7.6,125.0,15.0,159.0,16.4,3.0,...,False,True,False,False,False,True,False,False,True,False
4,2007,22.72,80.0,63.0,7.7,164.0,15.0,227.0,16.2,3.0,...,False,False,True,False,False,True,False,False,True,False


In [3]:
df_gd.shape

(5837, 107)

# Création jeu d'entraînement et jeu de test

In [4]:
from sklearn.model_selection import train_test_split

y=df_gd['Global_Sales']
X=df_gd.drop('Global_Sales', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)


# Normalisation

In [5]:
from sklearn.preprocessing import StandardScaler

# Normaliser les variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modele Lasso sans pipeline

In [6]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Créer une instance de la classe Lasso
modele_lasso = Lasso(alpha=0.1, max_iter=10000, random_state=42, fit_intercept=True)

# Entraîner le modèle sur l'ensemble d'entraînement
modele_lasso.fit(X_train, y_train)

# Évaluer la performance du modèle sur l'ensemble de test
score = modele_lasso.score(X_test, y_test)

# Faire des prédictions sur les données de test
y_pred_lasso = modele_lasso.predict(X_test)

# Évaluer les performances du modèle en utilisant les mesures de performance
r2 = r2_score(y_test, y_pred_lasso)
mse = mean_squared_error(y_test, y_pred_lasso)
mae = mean_absolute_error(y_test, y_pred_lasso)

print("R2 score:", r2)
print("MSE:", mse)
print("MAE:", mae)

R2 score: 0.21994516787607066
MSE: 1.6598683433042467
MAE: 0.6368472731984786


# Cross Validation

In [7]:
from sklearn.model_selection import cross_val_score, KFold

# Définir la stratégie de validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Effectuer la validation croisée sur le modèle avec la métrique R²
scores = cross_val_score(modele_lasso, X, y, cv=cv, scoring='r2')

# Afficher les scores de validation croisée
print("Scores de validation croisée: ", scores)

# Afficher la moyenne des scores de validation croisée
print("Score moyen de validation croisée: {:.2f}".format(scores.mean()))

Scores de validation croisée:  [0.19716191 0.21849303 0.10291017 0.15520674 0.14465589]
Score moyen de validation croisée: 0.16


## Test modèle LinearSVR

In [8]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# Création de la pipeline
pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    VarianceThreshold(threshold=0.1),
    LinearSVR(C=20.0, dual=True, epsilon=0.01, loss="squared_epsilon_insensitive", tol=0.001)
)

set_param_recursive(pipeline.steps, 'random_state', 42)

# Entraînement du modèle avec les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluation des performances du modèle sur les données de test
score = pipeline.score(X_test, y_test)
print(score)

0.28444220010715204


c:\Users\debor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Test modèle Lasso avec Pipeline1

In [9]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.linear_model import Lasso

# Création de la pipeline
pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    VarianceThreshold(threshold=0.1),
    Lasso(alpha=0.1, max_iter=10000, random_state=42, fit_intercept=True)
)

set_param_recursive(pipeline.steps, 'random_state', 42)

# Entraînement du modèle avec les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluation des performances du modèle sur les données de test
score = pipeline.score(X_test, y_test)
print(score)

0.37586370335447206


## Test modèle Lasso avec pipeline2

In [10]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso

# Création de la pipeline
pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    VarianceThreshold(threshold=0.1),
    StackingEstimator(estimator=RandomForestRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=14, min_samples_split=7, n_estimators=100)),
    Lasso(alpha=0.1, max_iter=10000, random_state=42, fit_intercept=True)
)

set_param_recursive(pipeline.steps, 'random_state', 42)


# Entraînement du modèle avec les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluation des performances du modèle sur les données de test
score = pipeline.score(X_test, y_test)
print(score)


0.4774210582058045


## Test modèle LassoLarsCV avec pipeline2

In [11]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LassoLarsCV

# Création de la pipeline
pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    VarianceThreshold(threshold=0.1),
    StackingEstimator(estimator=RandomForestRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=14, min_samples_split=7, n_estimators=100)),
    LassoLarsCV()
)

set_param_recursive(pipeline.steps, 'random_state', 42)


# Entraînement du modèle avec les données d'entraînement
pipeline.fit(X_train, y_train)

# Évaluation des performances du modèle sur les données de test
score = pipeline.score(X_test, y_test)
print(score)


0.48870372735710976


# Pipeline

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

df_clean=pd.read_csv("df_clean_v2.csv")


In [18]:
df=pd.get_dummies(df_clean)

In [19]:
from sklearn.model_selection import train_test_split

y=df['Global_Sales']
X=df.drop('Global_Sales', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectFwe
from sklearn.linear_model import LassoLarsCV
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LassoLarsCV

# Création de la pipeline 3
pipeline3 = make_pipeline(
    StandardScaler(),
    #MinMaxScaler(),
    SelectFwe(score_func=f_regression, alpha=0.009000000000000001),
    #StackingEstimator(estimator=DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=12)),
    StackingEstimator(estimator=RandomForestRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=14, min_samples_split=7, n_estimators=100)),
    VarianceThreshold(threshold=0.1),
    LassoLarsCV()
)

set_param_recursive(pipeline3.steps, 'random_state', 42)


# Entraînement du modèle avec les données d'entraînement
pipeline3.fit(X_train, y_train)

# Évaluation des performances du modèle sur les données de test
score = pipeline3.score(X_test, y_test)
print(score)

0.9350415607853348


In [22]:
# Faire des prédictions sur les données de test
y_pred_pipe = pipeline3.predict(X_test)

# Évaluer les performances du modèle en utilisant les mesures de performance
r2 = r2_score(y_test, y_pred_pipe)
mse = mean_squared_error(y_test, y_pred_pipe)
mae = mean_absolute_error(y_test, y_pred_pipe)

print("R2 score:", r2)
print("MSE:", mse)
print("MAE:", mae)

R2 score: 0.9350415607853348
MSE: 0.13822420225165116
MAE: 0.16193892023809917


Le R2 score, également appelé coefficient de détermination, mesure la proportion de la variance totale de la variable cible qui est expliquée par le modèle. Un score R2 de 0,94 indique que ce modèle explique 92% de la variance dans les données, ce qui est très élevé et suggère que ce modèle est capable de prédire la variable cible avec une grande précision.

Le MSE (mean squared error) mesure la moyenne des carrés des différences entre les prévisions du modèle et les valeurs réelles de la variable cible. Un MSE de 0,13 indique que, en moyenne, les prévisions de ce modèle diffèrent des valeurs réelles de la variable cible de 0,15 (la racine carrée du MSE) ce qui est relativement faible et suggère que ce modèle est assez précis.

Le MAE (mean absolute error) mesure la moyenne des différences absolues entre les prévisions du modèle et les valeurs réelles de la variable cible. Un MAE de 0,2 indique que, en moyenne, les prévisions de ce modèle diffèrent des valeurs réelles de la variable cible de 0,04, ce qui est également relativement faible et suggère que ce modèle est assez précis.

En somme, les scores que vous avez fournis indiquent tous que notre modèle est capable de prédire la variable cible avec une grande précision et une faible erreur.